In [22]:
!pip install h5py

In [23]:
pip install xarray h5netcdf h5py netcdf4



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install geopandas


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install rasterio


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
# ==========================
# 1. Imports et SSL
# ==========================
import ssl
import os
import warnings
import pandas as pd
import xarray as xr
import earthaccess
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

warnings.filterwarnings('ignore')
%matplotlib inline



In [27]:
# Deactivate SSL verification for Mac users 
import earthaccess, os

# Deactivate ssl in a fsspec session used by earthaccess
earthaccess.__store__.get_fsspec_session().kwargs['ssl'] = False
os.environ['FSSPEC_HTTP_VERIFY'] = 'False'

import certifi
import ssl
ssl_context = ssl.create_default_context(cafile=certifi.where())

In [28]:
# ==========================
# 2. Environment variables used for login
# ==========================
os.environ['EARTHDATA_USERNAME'] = 'tnourlyn'
os.environ['EARTHDATA_PASSWORD'] = 'Lutinette2411%'


In [29]:

# ==========================
# 3. EarthAccess authentification
# ==========================
try:
    auth = earthaccess.login(strategy="environment")
    print("✅ Authentification réussie !")
except Exception as e:
    print(f"❌ Échec de login : {e}")


✅ Authentification réussie !


In [33]:

# ==========================
# 4. Granules Search
# ==========================
minlon = 11.7 #Bouding box info for Nigeria -- To change depending on chosen country 
maxlon = 12.1
minlat = 13.0
maxlat = 13.4

granules = earthaccess.search_data(
    short_name='FLDAS_NOAH01_C_GL_M',
    temporal=("2020-01", "2024-12"),
    bounding_box=(minlon, minlat, maxlon, maxlat)
)
print(f"Granules found : {len(granules)}")


Granules trouvés : 60


In [34]:

# ==========================
# 5. Data Opening
# ==========================
# Liste de fichiers locaux ou URLs
fn_list = earthaccess.open(granules)

ds = xr.open_dataset(fn_list[0], engine='h5netcdf')
print(ds)


QUEUEING TASKS | : 100%|██████████| 60/60 [00:00<00:00, 3242.73it/s]
PROCESSING TASKS | : 100%|██████████| 60/60 [00:22<00:00,  2.71it/s]
COLLECTING RESULTS | : 100%|██████████| 60/60 [00:00<00:00, 163414.44it/s]


<xarray.Dataset> Size: 605MB
Dimensions:                 (time: 1, bnds: 2, Y: 1500, X: 3600)
Coordinates:
  * time                    (time) datetime64[ns] 8B 2020-01-01
  * Y                       (Y) float64 12kB -59.95 -59.85 ... 89.85 89.95
  * X                       (X) float64 29kB -179.9 -179.8 ... 179.9 180.0
Dimensions without coordinates: bnds
Data variables: (12/29)
    time_bnds               (time, bnds) datetime64[ns] 16B ...
    Evap_tavg               (time, Y, X) float32 22MB ...
    LWdown_f_tavg           (time, Y, X) float32 22MB ...
    Lwnet_tavg              (time, Y, X) float32 22MB ...
    Psurf_f_tavg            (time, Y, X) float32 22MB ...
    Qair_f_tavg             (time, Y, X) float32 22MB ...
    ...                      ...
    SoilMoi40_100cm_tavg    (time, Y, X) float32 22MB ...
    SoilMoi100_200cm_tavg   (time, Y, X) float32 22MB ...
    SoilTemp00_10cm_tavg    (time, Y, X) float32 22MB ...
    SoilTemp10_40cm_tavg    (time, Y, X) float32 22MB ...

Let's create the data frame thanks to the extracted data 